In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
import requests
import pickle

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [83]:
df_train=pd.read_csv("data/train_tweet.csv",index_col=False)
df_test=pd.read_csv("data/test_tweet.csv",index_col=False)
print(len(df_train),len(df_test))

27481 3534


In [84]:
df_train.columns

Index(['textID', 'text', 'selected_text', 'sentiment'], dtype='object')

In [85]:
df_train=df_train[['textID', 'text', 'sentiment']]

In [86]:
df_train.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [87]:
df_test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [90]:
df=df_train.append(df_test)
len(df)
df=df.dropna()

In [92]:
df["sentiment"]=df["sentiment"].astype('category')
df["sentiment_target"]=df["sentiment"].cat.codes

In [93]:
df.head()

,text,sentiment,sentiment_target
0,"I`d have responded, if I were going",neutral,1
1,Sooo SAD I will miss you here in San Diego!!!,negative,0
2,my boss is bullying me...,negative,0
3,what interview! leave me alone,negative,0
4,"Sons of ****, why couldn`t they put them on t...",negative,0


In [94]:
x=df["text"].values
y=df["sentiment_target"].values
len(y)

31014

In [95]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.8,random_state =0)

In [96]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=1000, decode_error="ignore")
vectorizer.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [97]:
#X_train_vectorized = vectorizer.transform(x_train)

In [98]:
from sklearn.naive_bayes import MultinomialNB
cls = MultinomialNB()
# transform the list of text to tf-idf before passing it to the model
cls.fit(vectorizer.transform(x_train), y_train)
 
from sklearn.metrics import classification_report, accuracy_score
 
y_pred = cls.predict(vectorizer.transform(x_test))
print(accuracy_score
      , y_pred))
print(classification_report(y_test, y_pred))

0.6230856037401258
             precision    recall  f1-score   support

          0       0.68      0.48      0.56      6971
          1       0.56      0.74      0.63     10089
          2       0.72      0.60      0.65      7752

avg / total       0.64      0.62      0.62     24812

